### Домашняя работа к уроку 10
### Студент: Абрамов А.В.

Разобраться с моделькой перевода (без механизма внимания) как она устроена, запустить для перевода с русского на английский (при желании можно взять другие пары языков)

In [23]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from keras.layers import GRU, Concatenate

import unicodedata
import re
import numpy as np
import os
import io
import time

In [11]:
path_to_file = "./rus-eng/rus.txt"

In [5]:
def preprocess_sentence(w):
    w = w.lower().strip()
  
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
  
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)
  
    w = w.strip()
  
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [6]:
preprocess_sentence("I can't go.")

"<start> i can't go . <end>"

In [12]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENG, RUS]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]
  
    return zip(*word_pairs)

In [13]:
en, ru = create_dataset(path_to_file, None)
print(en[0])
print(ru[0])

<start> go . <end>
<start> марш ! <end>


In [14]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
  
    tensor = lang_tokenizer.texts_to_sequences(lang)
  
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
  
    return tensor, lang_tokenizer

In [15]:
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset(path, num_examples)
  
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
  
    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [16]:
len(en), len(ru)

(487600, 487600)

In [17]:
# Try experimenting with the size of that dataset
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [18]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [19]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [20]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
4 ----> я
52 ----> хочу
9 ----> ,
106 ----> чтобы
10 ----> ты
26 ----> его
1125 ----> попробовал
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
4 ----> i
44 ----> want
7 ----> you
11 ----> to
107 ----> try
10 ----> it
3 ----> .
2 ----> <end>


In [21]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [22]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 15]), TensorShape([64, 11]))

In [24]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.Bidirectional(GRU(self.enc_units,
                                   return_sequences=False,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform'))
    

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state_frw, state_bcw = self.gru(x, initial_state = hidden)
        return state_frw, state_bcw

    # def initialize_hidden_state(self):
    #     return tf.zeros((self.batch_sz, self.enc_units))

    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_sz, self.enc_units)) for i in range(2)]

In [25]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
# sample input
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
sample_hidden = encoder.initialize_hidden_state()
forward_hidden, backward_hidden = encoder(example_input_batch, sample_hidden)
print('Encoder forward_h shape: (batch size, units) {}'.format(forward_hidden.shape))
print('Encoder backward_h shape: (batch size, units) {}'.format(backward_hidden.shape))

Encoder forward_h shape: (batch size, units) (64, 1024)
Encoder backward_h shape: (batch size, units) (64, 1024)


In [26]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(2*self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, encoder_fwd_state, encoder_back_state):
        # enc_output shape == (batch_size, max_length, hidden_size)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x, initial_state=Concatenate(axis=-1)([encoder_fwd_state, encoder_back_state]))

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state

In [27]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      forward_hidden, backward_hidden)

In [28]:
decoder_sample_x.shape

TensorShape([64, 7356])

In [29]:
decoder_sample_h.shape

TensorShape([64, 2048])

In [30]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
  
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
  
    return tf.reduce_mean(loss_)

In [34]:
checkpoint_dir = '/training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [35]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
  
    with tf.GradientTape() as tape:
        enc_hidden = encoder(inp, enc_hidden)
    
        forward_hidden, backward_hidden = enc_hidden
    
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
    
        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]): # Максимальная длина последовательности 
            # passing enc_output to the decoder
            predictions, dec_hidden = decoder(dec_input, forward_hidden, backward_hidden)
       
            # считаем лосс по каждому предсказанному слову
            loss += loss_function(targ[:, t], predictions)
      
            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)
  
    batch_loss = (loss / int(targ.shape[1]))
  
    variables = encoder.trainable_variables + decoder.trainable_variables
  
    gradients = tape.gradient(loss, variables)
  
    optimizer.apply_gradients(zip(gradients, variables))
  
    return batch_loss

In [36]:
EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()
  
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
  
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
    
        if batch % 100 == 0:
          print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                       batch,
                                                       batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)
  
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.2930
Epoch 1 Batch 100 Loss 0.2796
Epoch 1 Batch 200 Loss 0.3141
Epoch 1 Batch 300 Loss 0.2221
Epoch 1 Batch 400 Loss 0.2672
Epoch 1 Batch 500 Loss 0.2723
Epoch 1 Batch 600 Loss 0.2372
Epoch 1 Batch 700 Loss 0.3289
Epoch 1 Batch 800 Loss 0.2554
Epoch 1 Batch 900 Loss 0.2523
Epoch 1 Batch 1000 Loss 0.2246
Epoch 1 Batch 1100 Loss 0.1874
Epoch 1 Batch 1200 Loss 0.3320
Epoch 1 Loss 0.2781
Time taken for 1 epoch 1613.1543941497803 sec



In [38]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
  
    sentence = preprocess_sentence(sentence)
  
    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
  
    result = ''
  
    hidden = [tf.zeros((1, units)) for i in range(2)]
    enc_hidden = encoder(inputs, hidden)
  
    forward_hidden, backward_hidden = enc_hidden
  
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
  
    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, forward_hidden, backward_hidden)
    
        # storing the attention weights to plot later on
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '
    
        if targ_lang.index_word[predicted_id] == '<end>':
          return result, sentence
    
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)
  
    return result, sentence

In [39]:
def translate(sentence):
    result, sentence = evaluate(sentence)
  
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [41]:
translate('Я не боюсь.')

Input: <start> я не боюсь . <end>
Predicted translation: i'm not afraid . <end> 


In [42]:
translate('Здесь очень жарко.')

Input: <start> здесь очень жарко . <end>
Predicted translation: it's very hot here . <end> 


In [44]:
translate('Он приехал на коне.')

Input: <start> он приехал на коне . <end>
Predicted translation: he went by a walk . <end> 
